In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from matplotlib.patches import Arc
import numpy as np
from math import sqrt


In [4]:
class Ring:
    """
    Class representing a Cherenkov ring
    
    Unit radius, centred at (x, y) provided to the ctor

    """

    _x = 0.0
    _y = 0.0

    # Angle between the horizontal and the line joining the centre to the origin
    _alpha = 0.0

    # Distance to the origin
    _d = 0.0
    
    # Radius
    _radius = 1.0

    def __init__(self, x, y):
        self._x = x
        self._y = y
        self._alpha = np.arctan2(y, x)
        self._d = sqrt(x ** 2 + y ** 2)

    def _r(self, phi):
        """
        Returns the distance from the origin to the edge of the ring, looking along a line at an angle phi
        from the horizontal.

        phi in rad

        """
        # Our distance is the positive solution to the quadratic equation r^2 + br + c = 0
        # This just comes out of some geometry if you draw the triangles
        b = -2 * self._d * np.cos(phi - self._alpha)
        c = (self._d * self._d) - 1

        return (-b + sqrt(b * b - 4 * c)) / 2

    def misalignment(self, n):
        """
        Returns an array of n distances from the origin to the edge of the ring, equally spaced in angle
        between 0 and 2 pi

        Returns also an array of n array equally spaced between 0 and 2pi

        """
        angles = np.linspace(0, 2 * np.pi)

        # Could make this faster, don't need to
        return np.array([self._r(angle) for angle in angles]), angles

    def centre(self):
        return self._x, self._y
    
    def boundary(self, n):
        """
        Returns two n-length arrays (x, y) of x and y co-ordinates of the ring edges
        
        Not guaranteed to start at any particular angle - i.e. 
        
        """
        angles = np.linspace(0, 2*np.pi)
        return np.array([self._x + np.cos(x) for x in angles]), np.array([self._y + np.sin(x) for x in angles])

    def move(self, x, y):
        """
        Move this ring to be centred on (x, y)

        """
        self.__init__(x, y)
        

In [18]:
n = 25
My_Ring = Ring(0.0, 0.0)
distances, angles = My_Ring.misalignment(n)

fig, _ = plt.subplots(figsize=(9.0, 3.0))
ax = plt.subplot2grid((1, 3), (0, 0))
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax2 = plt.subplot2grid((1, 3), (0, 1), colspan=2)

circle_x, circle_y = My_Ring.boundary(n)
ring, = ax.plot(circle_x, circle_y, "k--")
ax.plot(0, 0, "r.", label="Tracking Spot")
ring_centre, = ax.plot(0, 0, "bx", label="Cherenkov Ring Centre")
ax.axis("off")
ax.legend(loc=(-0.5, 0.9))

# Mark an angle to illustrate what we're doing
# Use the 2n/5 th point around the circle to illustrate
horizontal, = ax.plot((0, circle_x[0]), (0, 0), "k")
not_horizontal, = ax.plot((0, circle_x[n//5]), (0, circle_y[n//5]), "k")  # name??
phi = np.arctan2(circle_y[n//5]/5, circle_x[n//5]/5)  # Angles that define an arc
r_label = ax.text(-0.2 + circle_x[n//5]/2, circle_y[n//5]/2, "r")
ax.text(0.1, 0.1, r"$\phi$")

line, = ax2.plot(angles, distances)
ax2.set_ylabel(r"$r\ (\phi)$")
ax2.set_ylim(0, 2)
ax2.set_yticks([])
x_tick = np.linspace(0, 2, 5)
ax2.set_xticks(x_tick * np.pi)
ax2.set_xticklabels([fr"${x}\pi$" for x in x_tick])
ax2.tick_params(direction="in")
ax2.text(np.pi, 0.1, r"$\phi\ /rad$", horizontalalignment="center")

ax_x_slider = plt.axes([0.05, 0.1, 0.3, 0.03])
x_slider = Slider(ax_x_slider, "x", -1.0, 1.0, 0.0)

ax_y_slider = plt.axes([0.05, 0.05, 0.3, 0.03])
y_slider = Slider(ax_y_slider, "y", -1.0, 1.0, 0.0)

def update(val):
    x = x_slider.val
    y = y_slider.val
    My_Ring.move(x, y)
    
    misalignment, _ = My_Ring.misalignment(n)
    line.set_ydata(misalignment)
    
    ring_coords = My_Ring.boundary(n)
    ring.set_data(*ring_coords)
    horizontal.set_xdata((0, misalignment[0]))
    not_horizontal.set_data((0, ring_coords[0][n//5]), (0, ring_coords[1][n//5]))
    r_label.set_position((-0.2+ring_coords[0][n//5]/2, ring_coords[1][n//5]/2))
    
    ring_centre.set_data(x, y)

x_slider.on_changed(update)
y_slider.on_changed(update)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
plt.close("all")